# Урок 4. 
## Парсинг HTML. XPath

1) Структура страницы HTML - DOM

2) Язык запросов Xpath

3) Парсинг HTML в Python - библиотека lxml

XPath - достаточно старый, борадатый инструмент. Этот язык запросов помогает выбирать нужные нам данные из голого чистого html. Предварительно этот код преобразуется к DOM. Точно такой же DOM как был в Beautiful_Soup.

DOM (от англ. Document Object Model - объектная модель документа) - программный интерфейс, позволяющий программам скриптам получать доступ к содержимому html документов, а также изменять их содержимое, структуру и оформление таких документов. 

типы узлов:
- документ
- элементы
- текстовые узлы
- комментарии
- атрибуты
- текст
- пространство имён
- инструкция обработки

В XPath  другой парсер в отличии от Beautiful_Soup. С другим механизмом работы. сами классы по другому называются, объекты. связи. Другие методы. 

Связи между объектами в доме схожи со связями Beautiful_Soup. Родительские, дочерние, родственные. 

Навигация по DOM 
```
document
    |
childNodes[0]   (<HTML>)
    |
childNodes[2]   (<BODY>)
    |
childNodes[1]   (<p>)
    |
childNodes[0]   (<input>)  
    |
value
```

Если нам необходимо выставить какую то цепочку зависимостей одного объекта от другого чтобы не запускать заново метод поиска, просто переходим по этим узлам от изночально корневого элемента, до нужного нам, используя связи по средствам обращения к каждому встречному узлу на пути от точки А до точки Б пока не дойдём до нужной нам части. 

In [ ]:
document.childNodes[0].childNodes[2].childNodes[1].childNodes[0].value

## Язык запросов XPath
XPath (XML Path Language) - Язык запросов к элементам XML-документа. Разработан для организации доступа к частям документа XML в файлах трансформации XSLT и является стандартом консорциума W3C. XPath призван реализовать навигацию по DOM в XML. в Xpath используется компактный синтаксис, отличный от принятого в XLM.

XML - стандарт формата данных (похожий на HTML, т.к и там и там есть теги). Используется в некоторых организациях. Некоторые API могут выдавать XML формат. 

#### Алгоритмы работы XPath
у нас есть некий HTML код:

In [ ]:
<html>
    <body>
        <div>Первый слой
            <span>блок текста в первом слое </span>
        </div>
        <div>Второй слой</div>
        <div>Третий слой
            <span class='text'>первый блок в третьем слое</span>
            <span class='text'>второй блок в третьем слое</span>
            <span>третий блок в третьем слое</span>
        </div>
        <span>четвёртый слой</span>
    </body>
</html>

Нас в данном коде интересуют 2 тега:
```
<span class='text'>первый блок в третьем слое</span>
<span class='text'>второй блок в третьем слое</span>
```

Это должен будет быть наш конечный результат. В принципе мы достигаем этой цели с ипоьзованием текущего Xpath пути. Пройдёмся по нему пошагово

#### Xpath путь: `/html/body/*/span[@class]`

будет соответствовать в нём двум элементам исходного документа.
```
<span class='text'>
<span class='text'>
```

Xpath  можно сравнить с путём к файлу в нашей операционной системе. 
Каждый узел есть директория, и в конечном счёте нужный нам элемент это файл с каким то расширением. Если проводить грубую анологию с ОС. 

Для того чтобы дойти до нужного файла, нам надо указать путь, относительно корня диска операционной системы или корня раздела. В целом так всё и происходит. 

Мы берём относительно корня. (корнем является document). Корень -  Базовый объект от которого наследуются все остальные объекты, являющиеся элементами дерева. Изначально весь парсинг происходит отталкиваясь от этого корня. Поэтому первым мы ставим "/". 

"/" в Xpath указывает отношение элемента справа к элементу слева. "/" указывает, что элемент который справа является непосредственно ребёнком элемента который слева. 

1. шаг `/html`  после него снова идёт одиночный слэш "/". 

2. шаг `/body` благодоря слэшу мы будем понимать что ищем детей тега html

3. шаг `/*` - звёздочка означает, что там может быть любой тэг. Всё что угодно. Следует применять с большой осторожностью. все дети body будут подходить под третий шаг.

4. шаг `/span[@class]` Мы явно задаём что у тэга span должен быть класс

Парсинг заканчивается когда доходим до закрывающего тэга `</html>`
Результатом работы будут 2 элемента, которые возвратятся списком. 
Рекурсивно обходит весь html без исключений. 

Результатом работы метода всегда будет список. Если элементов нет нам вернётся пустой список. 

### Язык запросов XPath

`выражение: messages` - выбирает все узлы с именем 'messages'

`выражение: /messages` - выбирает корневой элемент сообщений. (Если путь начинается с косой серты ("/") то он всегда представляет абсолютный путь к элементу

`выражение: messages/note` - выбирает все элементы 'note', являющиеся потомками элемента 'messages'

`выражение: //note`- выбирает все элементы 'note', независимо от того, где они находятся

`выражение: messages//note` -  выбирает все элементы 'note', являющиеся потомками элемента 'messages' независимо от того, где они находятся от элемента 'messages'

`выражение: //@date` - выбирает все атрибуты с именем 'date'


У  XPath есть дополнительные инструменты, котрые позволяют очень эффективно быстро без написания кода тестить пути. 

С помощью инструмента разработчика открываем html код ресурса, где надо данные. Мы можем к уже загруженному html применить какой нибудь селектор.  нажимаем сочитание клавиш `CTRL + F` и в открывшейся строчке (внизу страницы) ввести XPath селектор.  Это не нагружает сервер, не отправляет запрос на сервер, это работает с локальными данными html которые сейчас в браузере отображаются. Те элементы, которые попадают под селекцию в html коде подсвечиваются жёлтым и также подсвечиваются на странице. Это стандартные инструменты любого браузера. 

В браузере можно установить дополнительно chroPATH. ужно смотреть в настройках браузера дополнения (магазин расширений). Появится это расширение в панели разработчика. В окне со стилями, слоями и другими инструментами. 

#### Как работать с XPath

Предположим нам надо собрать топ 5 новостей по версии яндекса. 
Начинаем с анализа HTML кода. Нам нужны контейнеры, которые содержат необходимые нам данные.  Мы собираем 5 новостей и видим что они находятся каждый в своём контейнере и мы попробуем найти, что есть общее у этих контейнеров.  Навожу на нужный элемент мышкой, ПКМ, и выбираю copy -> xpath. 
```
//[@id="news_552134667988.6136"]/article/div[2]/ul/li[2]
```
Смотрим контейнер, где находится интересующая нас новость. 

`<li class="news-story__story-2M" data-testid="news-item"></li>`

У нас есть 5 таких контейнеров, в которых находится 5 новостей.  Нам должно быть удобно работать с этими тегами. Надо написать такой селектор, который бы выбрал эти элементы сразу на старнице.  Смотрим на внешний контейнер, где находятся интересующие нас контейнеры с новостями. Ищем родителя тэгов `<li class="news-story__story-2M"`

Это будет тег `<ul class="card-news__stories-Bu" style="height:auto"> </ul>`

пишем селектор xpath: //ul и проверяем его в открывшейся поисковой строке в инструментах разработчика `CTRL + F`. В результате жёлтым подсветился контейнер  `<ul class="card-news__stories-Bu" ` Без указания класса просто находим тэг `<ul>`. К классу привяжемся позже. 
Причём мы видим что он нашл 16 таких тэгов.  сейчас подсвечен 2 of 16.

/ - слэш задаёт соотношение элемента справа к элементу слева. Используя одиночный слэш, мы говорим, что это ребёнок элемента слева.  Используя двойной слэш // - мы говорим, что это потомок n-ого поколения.  Задача xpath ограничившись элементом слева найти внутри него потомка с указанными параметрами. В частности параметров у нас немного, это должен быть тэг `<ul>`. 

Нам нужно максимально отвязаться от вёрстки. Когда ставим два слэша - говорим ищи по всему документу. 

Когда у нас есть этот контейнер с нужной нам информацией, мы можем войти в этот тег. Смотрим, что у него есть дети `<li class="news-story__story-2M"`? в которых находится нужная нам информация. 
коллектор имеет теперь вид:`//ul/li` Ищем всех детей li внутри ul.

Зайдём в ребёнка и посмотрим где непосредственно находится нужная нам информация. 

```<span class="news-story__text-2H" aria-label="Заголовок">Число погибших в&nbsp;результате землетрясения в&nbsp;Турции увеличилось до&nbsp;284<!-- -->&nbsp;&nbsp;&nbsp;&nbsp;<!-- -->&nbsp;&nbsp;&nbsp;</span>
```

Анализируем уровни вложенности:
`ul -> li -> a -> div -> span `




Воспользуемся ещё раз двойным слэшем чтобы невилировать уровни вложенности. `//ul/li//span ` 

Если идти по структуре дерева, то коллектор будет иметь вид:
`//ul/li/a/div/span`

Если заменим например тег `<a>` на звёздочку, будет означать, что вместо звёздоки может быть любой тег `//ul/li/*/div/span`

например сколько тегов и узлов на странице с атрибутом класс `//*[@class]`. Можем более конкретно узнать сколько тегов div с атрибутом класс `//div[@class]`. Все div у которых есть атрибут но не знаем какой `//div[@*]` - вернутся все дивы у которых есть какой то атрибут. Также есть отрицание `//div[not(@)*]` . 

##### Уточняющие параметры
эти параметры пишутся в квадратных скобках `[ ]`. Если что то уточняет нашу выборку, это что то будет в квадратных скобках.  Можно указать индекс элемента. `//ul/li/a/div/span[1]`.  В xpath нумирация начинается не с 0, а с 1.  В xpath даже есть свои собственные функции. `//ul/li/a/div[position()<4]` например функция position. Можно добавлять логические операторы `//ul/li/a/div[position()<4 and position()>1]`. 

 # Практическая часть.
 
- собрать погоду с сайта яндекс. https://dzen.ru
- собрать 5 новостей с сайта яндекс https://dzen.ru
- собрать 5 новостей с новостного ресурса  например https://ria.ru

### Погода
Заходим на сайт в браузере и ищем элемент на странице, смотрим через инструмент разработчика его код. Нас интересует значение температуры. 
```
<div class="geoblock-weather__currentWeather-1P">-1<!-- -->°</div>
```
Дльше открываем поиск XPath `CTRL + F` и вводим:

`//div[@class='geoblock-weather__currentWeather-1P']`

Подводный камень заключается в том, что если у элемента более чем один класс, например:
`<div class="geoblock-weather__currentWeather-1P end_one_class">-1<!-- -->°</div>`

у нас 2 класса у тега div:
- geoblock-weather__currentWeather-1P
- end_one_class

поиск в XPATH по селектору `//div[@class='geoblock-weather__currentWeather-1P']` ничего не даст, потому что не умеет различать классы. Чтобы XPATH понял надо вводить атрибут класс целиком, он не разделяет класссы по пробелу.

##### Рассмотрим функцию contains
`//div[contains()]` передаём 2 параметра:
- значение узла в котором мы будем искать (атрибут, класс)
- значение этого самого атрибута

`//div[contains(@class, 'end_one_class')]` 

привязываемся только к одному классу. Если он эту последовательность символов находит, он возвращает результат. 
```
 <div class="geoblock-weather__currentWeather-1P end_one_class" data-tid="dehddjwx">-1<!-- -->°</div>
```
Предположим что у элемента есть ещё атрибуты и можем искать ширше:

`//div[contains(@class, 'end_one_class' and @data-tid)]`

или ещё ширше, возьмём тег выше `<main>`:

`//main/div[contains(@class, 'end_one_class' and @data-tid)]`

ещё накрутим какое нибудь условие:

`//main/div[contains(@class, 'end_one_class') and @data-tid and position()>3]`

Мы выстраиваем наш XPATH, чтобы он не привязывался к динамическим классам. 

`//main/div[contains(@class, 'end_one_class') and @data-tid and position()>3 and not(position()=last())]`

Position() - определяет какие теги по счёту. Из списка найденных тегов. Ограничение позиции выбрасывает лишние найденные элементы.  


Иногда нужные нам элементы находятся на одной странице но в разных блоках и мы одним XPATH не можем их выбрать. В данной ситуации можно объединить 2 XPATH в одно выражение поставив объединяющее или - " | ".  

`//body/note | //body/day` - выбираем в документе body 

`//body | //div` - выбираем во всём документе

Результат работы будет один список объектов полученных от 2 икс пасов. 
Получить один список, который потом надо будет обработать. 

# ПРАКТИЧЕСКИЙ ПРИМЕР

Начинаем мы с установки модуля. Парсер который позволяет получать доступ к xpath селекторам и с их помощью обрабатывать ДОМ. 

In [1]:
! pip install lxml

     ---------------------------------------- 3.9/3.9 MB 625.7 kB/s eta 0:00:00


Подключаем модуль отвечающий за html и остальные необходимые части.

In [58]:
from lxml import html
import requests
from pprint import pprint

Создаём ссылку на которую мы сейчас пойдём

In [ ]:
url_ebay = ''
url_ya = ''
url_ria = ''

выбираем категорию товара которую будем искать. Комплектующие для компьютера. Зашли на страницу входа список товаров, комплектующих. Эта страничка будет точкой входа и скопируем её себе.

In [59]:
url_ebay = 'https://www.ebay.com/b/Computer-Parts-Components/175673/bn_1643095'

Юзер агент критически обязателен! Когда используем икс пас работаем с вёрсткой, которая была непосредственно выдана браузеру. В каком браузере работаем его юзер агента и используем. В браузере `chrom://version` 

In [60]:
headers = {'user agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}

Параметров в ссылке нет - больше заголовки не заполняем, есть - можно указать в словаре headers

А дальше делаем запрос

In [61]:
response = requests.get(url_ebay, headers=headers)

In [62]:
response

<Response [400]>

Мы должны дальше из response получить DOM. Вызовим метод `fromstring()` а внутрь передаём хтмл код который содержится внутри нашего ответа

In [ ]:
dom = html.fromstring(response.text)

Дальше работать непосредственно с вёрсткой. Результатом работы икс пас является список. Поэтому в половине случаев хорошо использовать иной подход по сбору данных. Получим сразу списком наиминование всех товаров. исследуем страничку, кликнем по наименованию товара и посмотрим на то где он находится в коде. 

`<h3 class="s-item__title">Crucial 16GB 8GB 4GB 2Rx8 PC3L-12800S DDR3-1600MHz SODIMM Laptop Memory GS</h3>`

вызываем в инструментах разработчика `CTRL + F` xpath строку и выбираем селектор `//h3`
результат поиска 58 элементов но при анализе смотрим что первые 10 нам не нужны. Можем использовать класс дополнительно для селектора `//h3[@class="s-item__title"]` и вот уже в результате поиска 48 нужных нам штук. 

`//h3[position()>10]` - не сработает, так как ищут детей. Разный родитель и результат поиска ноль. 

теперь получаем список товаров, используя икс пас селектор

In [ ]:
names_stock = dom.xpath("//h3[@class='s-item__title']")

Попробуем изменить точку входа. Просто в поиске пишем наименование товара, которое будем искать. Например в поисковой строке ибея ввожу: `fluke` и отправляюсь на новую точку входа. 

In [70]:
url_ebay = 'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2380057.m570.l1313&_nkw=fluke&_sacat=0'

headers = {'user agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}
response = requests.get(url_ebay, headers=headers)

response

<Response [400]>

Получаем ДОМ 

In [ ]:
dom = html.fromstring(response.text)

Из Дома дальше с помощью селектора находим все узлы нам нужные (блоки, теги) и считываем с них информацию - текст. Получаем список. 

In [ ]:
markets = dom.xpath("//h3[@class='s-item__title']/text()")

Дальше мы хотим собрать например ссылку. 
```
<a data-interactions="[{&quot;actionKind&quot;:&quot;NAVSRC&quot;,&quot;interaction&quot;:&quot;wwFVrK2vRE0lhQQ0MDFHUlkzU1BaWUM5UEQ0SlYzMDZGQjNNR0o0MDFHUlkzU1BGRlZNMENCVDVaMUQwUDA1MkMAAAg3NDAwDE5BVlNSQwA=&quot;}]" target="_blank" data-s-jufo690="{&quot;eventFamily&quot;:&quot;LST&quot;,&quot;eventAction&quot;:&quot;ACTN&quot;,&quot;actionKind&quot;:&quot;NAVSRC&quot;,&quot;actionKinds&quot;:[&quot;NAVSRC&quot;],&quot;operationId&quot;:&quot;2351460&quot;,&quot;flushImmediately&quot;:false,&quot;eventProperty&quot;:{&quot;$l&quot;:&quot;138092160120784&quot;}}" _sp="p2351460.m1686.l7400" class="s-item__link" href="https://www.ebay.com/itm/255966678916?hash=item3b98cd8f84:g:izUAAOSwundj4atr&amp;amdata=enc%3AAQAHAAAA0C6UkOD5JwlyEjIHm3BG3eeyDBIMUdzg%2FzqFTf%2Bgrt5ZGPL8BoL%2FgDnYFdJhbtbWOHkCYYL%2BGrjKIjD54fFqQmzESh%2FKNf1MJbAhlZg6CIfQ%2FyILQISb9JAQs%2B4ZvDqydPtb6Tm3QLLJCSkBiUb0A3OH6GuccERuzC3XgwnQZGAAivxsR5XEKwtID1xvNUi0T1qkAQLjlq3RiLUJUgexB6JGT3Fw4rY3ucT7XUcbQa%2BbmTjRIfRppCoCRVege1VLuviNnkWqnN%2FRGW5DPnbLlUM%3D%7Ctkp%3ABk9SR_7n5sPHYQ"><div class="s-item__title"><span role="heading" aria-level="3">Fluke 54 II B термометр</span></div><span class="clipped">Открывается в новом окне или вкладке</span></a>
```

вот она в коде спряталась `href="https://www.ebay.com/itm/255966678916?`. 

Для демонстрации возможностей икс паса продемонстрируем следующий подход. В теге `<a>` есть класс `class="s-item__link"`. 
в икс пас поиске в панеле разработчиков собираем теги просто `//h3[@class='s-item__link']`. 

Обратим внимание что ранее используемый тег `<h3>` и `<a>` находятся близко друг к другу. мы можем в принципе тег `<a>`  взять отталкиваясь от тега `<h3>`.  Берём ранее использованный селектор от которого надо будет подняться до родителя `//h3[@class='s-item__title']` и поднимаемся до родителя используя синтаксис '/..' : `//h3[@class='s-item__title'/..]`  и в \том контейнере нас интересует значение атрибута `href`: `//h3[@class='s-item__title'/../@href]`

In [ ]:
links = dom.xpath("//h3[@class='s-item__title'/../@href]")

можно было через класс

In [ ]:
links = dom.xpath("a[@class='s-item__link'/@href]")

Подняться до дедушки: `//h3[@class='s-item__title'/../..]` 

Используя икс пас мы формируем списки данных. И потом эти списки мы потом сольём. 

Теперь возьмём цену. Бурём для этого `<span class='s-item__price'>` Но мы сталкиваемся с тем, что объектов больше чем нам нужно `//span[@class='s-item__price']` , так как есть некоторые товары с аукционной стоимостью и там указаны 2 цены. 

In [ ]:
prices = dom.xpath("span[@class='s-item__price']/text()")

Если links и markets у нас одинаковые по количеству элементов - 61, то prices нам вернулся список в 71 элемент. Потому что есть товары разной комплектации и цена указано от и до диапазоном.

Но кажется собран не весь текст, пробуем так использовать для текста двойной слэш :

In [ ]:
prices = dom.xpath("span[@class='s-item__price']//text()")

результат нам приходит список в 91 позицию. Когда мы используем выборку этого элемента, контейнера с классом `span[@class='s-item__price']` и прописывая один слэш текст - мы ищем текст среди детей. Но во вложенных узлах мы текста не соберём. Цена в разных элементах на разных уровнях вложенности. Где то цена на уровне ребёнка, а где то на уровне внуков. Двойной слэш поэтому используем, чтобы собрать по всем уровням вложенности тексты. Там где разная комплектация на один товар приходит 4 единицы в списке: 'от', '100', 'до', '200'. Поэтому длинный список надо теперь обработать после сбора данных. 

Например  'от', '100', 'до', '200' объединить в один элемент списка, чтобы для начала добиться совпадение длины списков. Идём по списку и если встречаем 'от' то три следующих элемента аппендим к нашему списку, делаем join по пробелу и делаем в одну строчку. Со структурой щас заморачиваться не будем - это постобработка, а щас надо собрать данные, чтобы они потом во едино слились. Длины списков нужны равные. 

Соберём ещё информацию о ссылке на отзывы. И начинаем всё с инспекции элемента в инструментах разработчика. 

ссылочка находится во внуках тега `<div class='s_item'>` спускаемся к детям `//div[@class='s_item'/a/href]`  - ссылки на отзывы. Получили список ссылок в 24 объекта. А остальные списки иные по размеру. 

Подход показанный выше не всегда эффективен. Не везде применим. В данной ситуации эффективний получать контейнеры с объявлениями и проходить их в цикле. обрабатывая каждый контейнер. На одной итеррации цикла в рамках одного контейнера мы получаем набор данных по одному товару и сформировать словарь по одному товару. Если каких то данных нету мы будем ставить наны. 

Для начала получим с помощью икс пас кнтейнер с данными по товару. 

In [ ]:
items = dom.xpath(//li[contains(@class, 'item_lin')])

Получили контейнеры с объявлениями и уже в цикле перебираем их. Причём если есть лишний контейнеры можем их исключить взяв срез. `item_info` - создадим словарик внутри цикла в который будем добавлять данные по одному товару. В качестве ДОМа передаём item.  `item_info` заполняем по соответствующим ключам. И потом добавляем всё в итоговый список `list_items.append(item_info)`. 

In [ ]:
list_items = []

for item in items[1:]:
    item_info = {}
    price = item.xpath("//span[@class='s-item__price']//text()")
    link = item.xpath("//a[@class='s-item__link'/@href]")
    market = item.xpath("//h3[@class='s-item__title']/text()")
    
    item_info['name'] = name
    item_info['price'] = price
    item_info['link'] = link
    item_info['market'] = market
    
    list_items.append(item_info)
    
pprint(list_items)

"CTRL" + D - позволяет скопировать строчку, которая выше. в пайчарме.

Результат получился отличным от ожидаемого. Икс пас - такой товарищь...Ему пофиг что происходит снаружи. Когда он видит селектор `//span[@class='s-item__price']//text()` и видит объект к которому его прмменять `item`. Он игнорирует его рамки. Он выходит наружу до корня и начинает применять селектор сверх до низу.  Чтоби икс пас работал в рамках одного объекта используем магический символ. Ставим точку "." в начале икс паса, говоря о том, что наш путь теперь не обсалютный, а относительный. `.//span[@class='s-item__price']//text()` Точка - символ в операционной системе, который указывает на текущую директорию. 

Результат тот который нужен получился. 
Дальше постобработка если необходима. Отдельной функцией к каждому элементу списка чистим и обрабатываем данные как надо.  

./modul.py - значит что мы ищем modul.py в текущей папке. указываем относительный путь и при переносе проекта файл найдётся, а абсолютный путь сломается. 

Дальше всё зависит от поставленной задачи и от той структуры которую хотим получить. Постобработкой данных. Хорошо подойдёт Монго. 

### собрать 5 новостей с сайта яндекс https://dzen.ru
Ищу контейнер со всеми пятью новостями через инструмент разработчика: `<ul class='card-news__stories-Bu'>` и через селектор в XPATH: `//ul[@class='card-news__stories-Bu']`


берём первую новость и вторую новость через XPATH. Объединяем 2 XPATH:
`//ul/li[1] | //ul/li[2]`

In [27]:
from lxml import html
import requests
from pprint import pprint

url_ya = 'https://dzen.ru/'
headers = {'User Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}

response = requests.get(url_ya, headers=headers)

In [28]:
response

<Response [400]>

In [ ]:
dom = html.fromstring(response.text)

В результате мы получаем объекты. И у каждого этого объекта имеются свои параметры. и нужный нам текст находится в одном из параметров. Чтобы получить нужный нам текст, дальше надо парсить до конца. 

Есть контейнер - это тег. Внутри этого тега есть информация. По сути контейнеры - это опорные элементы по пути к данным. Нам нужны сами данные. Так как по концепции дома - всё есть узел, то и текст и атрибуты тоже узлы нашего дерева. Когда простраиваем икс пас можем дойти до внутреннего узла выбранного нами объекта.  В частности мы говорим что хотим собрать все теги `<ul>` с классом `'card-news__stories-Bu'`:

`//ul[@class='card-news__stories-Bu']`

Да мы собираем теги но по сути нам нужна информация из этих тегов. Чтобы извлечь текстовые узлы из конечного результата нам нужно сделать ещё шаг от нашего объекта - добавить '/' и будем искать среди детей текст. 

`//ul[@class='card-news__stories-Bu'/text()']`



XPath позволяет решать более комплексные задачи более простыми вещами. Более тактический инструмент в плане постройки селекторов для выбора элементов страницы. 

Также есть CSS селлекторы. Они рассматриваются на другом курсе. Эти селекторы предназначены для выборки элементов.  

### Файл с урока

In [ ]:
from lxml import html
import requests
from pprint import pprint

url = 'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2380057.m570.l1313&_nkw=iphone&_sacat=0'
headers = {'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36'}

with open('./test.py') as f:
    pass

response = requests.get(url, headers=headers)

dom = html.fromstring(response.text)

items = dom.xpath("//li[contains(@class,'s-item')]")

list_items = []
for item in items[1:]:
    item_info = {}
    name = item.xpath(".//h3[@class='s-item__title']/text()")
    link = item.xpath(".//h3[@class='s-item__title']/../@href")
    price = item.xpath(".//span[@class='s-item__price']//text()")
    review = item.xpath(".//div[@class='s-item__reviews']/a/@href")


    item_info['name'] = name
    item_info['link'] = link
    item_info['price'] = price
    item_info['review'] = review
    list_items.append(item_info)

pprint(list_items)

# Разбор домашней работы

### https://github.com/belkanov

In [ ]:
# https://github.com/belkanov

"""
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность)
с сайтов HH(обязательно) и/или Superjob(по желанию).
Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы).
Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение).
Структура должна быть одинаковая для вакансий с обоих сайтов.
Общий результат можно вывести с помощью dataFrame через pandas.
Сохраните в json либо csv.
"""
import re
from time import sleep

import requests
from bs4 import BeautifulSoup as bs
from collections import namedtuple
import logging
import json

RE_SALARY = re.compile(r'(?:\d+\s*){1,}')
RE_CURRENCY = re.compile(r'\D+')
MAIN_URL = 'https://hh.ru'
VACANCY_URL = f'{MAIN_URL}/search/vacancy'
HEADERS = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'
}
vacancy_name = 'python'
PARAMS = {
    'text': vacancy_name
}

Salary = namedtuple('Salary', (
    'min',
    'max',
    'currency'
))

logging.basicConfig(format='%(asctime)s | %(levelname)-8s | %(name)s | %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger('job_scraper')
logger.setLevel(logging.INFO)

# если включить - можно увидеть редиректы
# requests_log = logging.getLogger("urllib3")
# requests_log.setLevel(logging.DEBUG)
# requests_log.propagate = True


def get_response(url, headers, params=None):
    timeouts = (5, 5)  # conn, read
    response = requests.get(url,
                            headers=headers,
                            params=params,
                            timeout=timeouts)
    if response.ok:
        # на случай редиректа hh.ru -> rostov.hh.ru
        splitted_response_url = response.url.split('/')
        new_main_url = f'{splitted_response_url[0]}//{splitted_response_url[2]}'
        return response, new_main_url


def get_int(re_match):
    return int(re_match.group().replace(' ', ''))


def get_salary(tag):
    if tag is None:
        return Salary(None, None, None)

    text = clear_tag_text(tag)
    # получалось неплохо через всякие сплиты/слайсы/джоины,
    # но потом пришли 'бел. руб.' и все сломалось =)
    # поэтому регулярки
    if 'от' in text:
        re_salary = RE_SALARY.search(text)
        salary = get_int(re_salary)
        re_currency = RE_CURRENCY.search(text, re_salary.end())
        return Salary(salary, None, re_currency.group())
    elif 'до' in text:
        re_salary = RE_SALARY.search(text)
        salary = get_int(re_salary)
        re_currency = RE_CURRENCY.search(text, re_salary.end())
        return Salary(None, salary, re_currency.group())
    else:
        re_min_salary = RE_SALARY.search(text)
        min_salary = get_int(re_min_salary)
        re_max_salary = RE_SALARY.search(text, re_min_salary.end())
        max_salary = get_int(re_max_salary)
        re_currency = RE_CURRENCY.search(text, re_max_salary.end())
        return Salary(min_salary, max_salary, re_currency.group())


def clear_tag_text(tag):
    text = tag.getText()
    text = text.replace('\n', '')
    # внезапно вылезло много пробелов
    splitted = [word for word in text.split() if word]
    text = ' '.join(splitted)
    return text


def parse_vacancy_link(tag):
    link = tag.get('href')
    return f'{MAIN_URL}{link}'


def save_to_file(data, file_name):
    with open(file_name, 'w', encoding='utf8') as f:
        f.write(data)


def parse_response(response, main_url):
    vacancies_info = []

    soup = bs(response.text, 'html.parser')
    anchor = soup.find('div', {'class': 'vacancy-serp-content'})

    vacancy_results = anchor.find('div', {'data-qa': 'vacancy-serp__results'})
    vacancies = vacancy_results.find_all('div', {'class': 'vacancy-serp-item'})
    logger.info('Нашел %d вакансий. Обрабатываю...', len(vacancies))
    for vacancy in vacancies:
        title_tag = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
        salary_tag = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})

        vacancy_info = {
            'name': clear_tag_text(title_tag),
            'salary': get_salary(salary_tag),
            'link': title_tag.get('href'),
            'site': main_url,
        }
        vacancies_info.append(vacancy_info)

    return vacancies_info, anchor


def main():
    all_vacancies = []

    page_cnt = 1
    url = VACANCY_URL
    headers = HEADERS
    params = PARAMS
    while True:
        logger.info('Parse page #%d', page_cnt)
        response, main_url = get_response(url, headers=headers, params=params)
        if not response:
            logger.error('NO response from %s', url)
            raise SystemExit(1)

        try:
            vacancies_info, anchor = parse_response(response, main_url)
        except ValueError as e:
            logger.exception(e)
            save_to_file(response.text, 'error_response.html')
            raise SystemExit(1)

        all_vacancies.extend(vacancies_info)

        logger.info('Ищу следующую страницу...')
        next_link = anchor.find('a', {'data-qa': 'pager-next'})
        if next_link:
            logger.info('Нашел.')
            url = f'{main_url}{next_link.get("href")}'
            params = None
            page_cnt += 1
            sleep(1)  # не будем спамить запросами
        else:
            logger.info('Видимо это последняя =) Всего обработано %d страниц',
                        page_cnt)
            break

    with open('vacancies.json', 'w', encoding='utf-8') as f:
        json.dump(all_vacancies, f, ensure_ascii=False)


if __name__ == '__main__':
    logger.info('--- START')
    main()
    logger.info('--- END')

Добавление монго 

In [ ]:
# https://github.com/belkanov

"""
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и
реализовать функцию, которая будет добавлять только новые вакансии в вашу базу.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с
заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).
Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов
с рейтингом не ниже введенного или качеством не ниже введенного
(то есть цифра вводится одна, а запрос проверяет оба поля)
"""
import logging
from pprint import pprint
from time import sleep
from typing import Optional

import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
from pymongo.collection import Collection
from pymongo.errors import DuplicateKeyError
from bson.objectid import ObjectId

from constants import *

logging.basicConfig(format='%(asctime)s | %(levelname)-8s | %(name)s | %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger('job_scraper')
logger.setLevel(logging.INFO)

# если включить - можно увидеть редиректы
# requests_log = logging.getLogger("urllib3")
# requests_log.setLevel(logging.DEBUG)
# requests_log.propagate = True


def get_response(url, headers, params=None):
    timeouts = (5, 5)  # conn, read

    for i in range(5):
        response = requests.get(url,
                                headers=headers,
                                params=params,
                                timeout=timeouts)
        if response.ok:
            logger.debug('response - OK')
            break
        else:
            logger.debug('response - NOT OK (%s)', response.status_code)
            sleep_time = i + 1
            logger.warning(f'Не смог получить ответ для %s. Подождем %d сек.',
                           response.url,
                           sleep_time)
            sleep(sleep_time)
    else:
        raise SystemExit(1, f'Так и не смог получить ответ для {response.url}')

    # на случай редиректа hh.ru -> rostov.hh.ru
    splitted_response_url = response.url.split('/')
    new_main_url = f'{splitted_response_url[0]}//{splitted_response_url[2]}'
    return response, new_main_url


def get_int(re_match):
    return int(re_match.group().replace(' ', ''))


def get_salary(tag):
    if tag is None:
        return Salary(None, None, None)

    text = clear_tag_text(tag)
    # получалось неплохо через всякие сплиты/слайсы/джоины,
    # но потом пришли 'бел. руб.' и все сломалось =)
    # поэтому регулярки
    if 'от' in text:
        re_salary = RE_SALARY.search(text)
        salary = get_int(re_salary)
        re_currency = RE_CURRENCY.search(text, re_salary.end())
        return Salary(salary, None, re_currency.group())
    elif 'до' in text:
        re_salary = RE_SALARY.search(text)
        salary = get_int(re_salary)
        re_currency = RE_CURRENCY.search(text, re_salary.end())
        return Salary(None, salary, re_currency.group())
    else:
        re_min_salary = RE_SALARY.search(text)
        min_salary = get_int(re_min_salary)
        re_max_salary = RE_SALARY.search(text, re_min_salary.end())
        max_salary = get_int(re_max_salary)
        re_currency = RE_CURRENCY.search(text, re_max_salary.end())
        return Salary(min_salary, max_salary, re_currency.group())


def clear_tag_text(tag):
    text = tag.getText()
    text = text.replace('\n', '')
    # внезапно вылезло много пробелов
    splitted = [word for word in text.split() if word]
    text = ' '.join(splitted)
    return text


def parse_vacancy_link(tag):
    link = tag.get('href')
    return f'{MAIN_URL}{link}'


def save_to_file(data, file_name):
    with open(file_name, 'w', encoding='utf8') as f:
        f.write(data)


def parse_response(response, main_url):
    vacancies_info = []

    soup = bs(response.text, 'html.parser')
    anchor = soup.find('div', {'class': 'vacancy-serp-content'})

    vacancy_results = anchor.find('div', {'data-qa': 'vacancy-serp__results'})
    vacancies = vacancy_results.find_all('div', {'class': 'vacancy-serp-item'})
    logger.info('Нашел %d вакансий. Обрабатываю...', len(vacancies))
    for vacancy in vacancies:
        title_tag = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})
        salary_tag = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})

        vacancy_info = {
            'name': clear_tag_text(title_tag),
            'salary': get_salary(salary_tag),
            'link': title_tag.get('href'),
            'site': main_url,
        }
        vacancies_info.append(vacancy_info)

    return vacancies_info, anchor


def get_mongo_collection(collection_name: str) -> Optional[Collection]:
    client = MongoClient(host=MONGODB_HOST,
                         port=MONGODB_PORT,
                         username='gb_mongo_root',
                         password='gb_mongo_root_pass')
    db = client[MONGODB_DB_NAME]

    collection = getattr(db, collection_name, None)
    if collection is None:
        raise ValueError(1, f"Коллекция {collection_name} не найдена")

    return collection

# ***************************************************************************
# Комментарий преподавателя по функции ниже:

# Придя на сайт хед хантер видим в названии ссылки присутствует некое числовое значение - 54960009
# Это айдишник который даётся в объявлении с вакансией. 
# Уникальный у каждого объявления. Хед хантер управляет объектами с помощью айдишников
# Это значение можно брать в качестве своего значения ай ди для документа
# _id = ObjectId(f'{vacancy_id:0>24}')
# В итоге 2 одинаковые вакансии к нам в базу не попадут. 
# Объект значения ай ди в монго может быть любого типа. кроме словарей. 
# Здесь к этому объекту приводят ObjectId(f'{vacancy_id:0>24}')

# Для создания уникального поля ай ди у каждой вакансии уникальная ссылка
# Можно взять эту ссылку и прогнать через функцию из библиотеки hashlib
# она формирует последовательности для поданных данных. 
# И для одинаковых данных она сформирует одинаковые последовательности
# можно ссылку в хэш перевести и этот хэш взять уникальным айдишником.

# Монго проверяет на дубли 
# except DuplicateKeyError as e:

# ***************************************************************************






def save_to_mongo(data):
    hh_vacancies = get_mongo_collection('hh_vacancies')
    # вероятно это не самый эффективный вариант вставки большого кол-ва данных
    # (по аналогии с одиночной вставкой в обычном SQL)
    # наверное, лучше бы было обогатить ИДшниками data и вставлять через insert_many...
    # хз - надо гуглить
    # но так проще работать с уже существующими вакансиями
    for row in data:
        # скорее всего ИД уникален только в рамках города (домена) - надо тестить
        # пока норм. всегда можно переделать =)
        try:
            # https://city.hh.ru/vacancy/54960009?fro...
            # -> 54960009
            vacancy_id = row['link'].split('/')[4].split('?')[0]
            # mongo любит только 24-знаковые ИД
            # может это и не best practice, зато не надо новый индекс делать =)
            _id = ObjectId(f'{vacancy_id:0>24}')
            hh_vacancies.insert_one({'_id': _id, **row})
        except DuplicateKeyError as e:
            logger.debug('Для ID=%s уже есть запись. Пропускаем', vacancy_id)  # noqa
            pass

        
        
# ***************************************************************************
# Комментарий преподавателя по функции ниже:

# Оператор '$elemMatch': Он берёт значение 'salary' и сравнивает 
# со всеми значениями salary_value - списка состоящего из зарплат. 
# и если '$elemMatch' пропусая значение 'salary' по всему списку 'salary_value'
# находит элементы большие или равные '$gte'
# данный документ вернётся в качестве результата поиска
# 
# ***************************************************************************

def filter_vacancies_by_salary(salary_value):
    sleep(1)  # дадим логам отлежаться, а то может получиться каша при выводе
    print(f'--- Вакансии с ЗП {salary_value:,}+')
    hh_vacancies = get_mongo_collection('hh_vacancies')
    # запросы конечно смотрятся страшно на фоне обычных SQL =)
    for row in hh_vacancies.find({
        # либо у нас есть значение больше искомого
        '$or': [{'salary': {'$elemMatch': {'$gte': salary_value}}},
                {
                    # либо у нас указана только начальная зарплата
                    '$and': [
                        {'salary.0': {'$ne': None}},
                        {'salary.1': None},
                    ]
                }
                ]
    }):
        pprint(row)


def main():
    page_cnt = 1
    url = VACANCY_URL
    headers = HEADERS
    params = PARAMS
    while True:
        logger.info('Parse page #%d', page_cnt)
        response, main_url = get_response(url, headers=headers, params=params)
        if not response:
            logger.error('NO response from %s', url)
            raise SystemExit(1)

        try:
            vacancies_info, anchor = parse_response(response, main_url)
        except ValueError as e:
            logger.exception(e)
            save_to_file(response.text, 'error_response.html')
            raise SystemExit(1)

        logger.info('Сохраняю их в Mongo')
        save_to_mongo(vacancies_info)

        logger.info('Ищу следующую страницу...')
        next_link = anchor.find('a', {'data-qa': 'pager-next'})
        if next_link:
            logger.info('Нашел.')
            url = f'{main_url}{next_link.get("href")}'
            params = None
            page_cnt += 1
            sleep(1)  # не будем спамить запросами
        else:
            logger.info('Видимо это последняя =) Всего обработано %d страниц',
                        page_cnt)
            break

    filter_vacancies_by_salary(100_000)


if __name__ == '__main__':
    logger.info('--- START')
    main()
    logger.info('--- END')


### https://github.com/Androkotey

In [ ]:
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
from contextlib import contextmanager

# мои модули
from parser_hh import get_vacancies
from mongo_queries import salary_filter


# Будем хранить ссылки на вакансии в множестве
PRIMARY_KEYS = set()
# заменить проверку на
# if collection.find_one('link', doc['link']):
#     print('Документ существует в базе')
# Но лучше сделать update_one с параметром upsert=True


def pprint_cursor_object(cursor):
    for doc in cursor:
        pprint(doc)


@contextmanager
def connect_to_mongodb_collection(database_name, collection_name, delete=False):
    """ Открывает соединение, удаляет коллекцию после отработки, если необходимо, и закрывает соединение """

    client = MongoClient('127.0.0.1', 27017)
    db = client[database_name]
    collection = db[collection_name]

    yield collection

    count_documents = collection.count_documents({})
    if delete:
        db.drop_collection(collection_name)
        print(f"Удалена коллеция с {count_documents} документами")
    client.close()


def add_data_to_collection(collection, data):
    """ Добавляет вакансии в коллекцию """

    for doc in data:
        try:
            if doc['link'] in PRIMARY_KEYS:
                raise DuplicateKeyError('err')
            PRIMARY_KEYS.add(doc['link'])
            collection.insert_one(doc)
        except DuplicateKeyError:
            print(f'Вакансия {doc["name"]} в {doc["address"]} уже существует в базе')


def get_sample(collection, num=1):
    """ Выбирает несколько записей из коллекции и убирает поля _id (чтобы честно по ссылке проверять дубликаты)"""

    return collection.aggregate([{'$sample': {'size': num}}, {'$project': {"_id": 0}}])


def main():
    with connect_to_mongodb_collection(database_name='hw3_database',
                                       collection_name='vacancies',
                                       delete=True) as vacancies:
        list_of_vacancies = get_vacancies(text='data scientist')  # получили вакансии от парсера
        add_data_to_collection(vacancies, list_of_vacancies)  # добавили вакансии в базу
        sample = get_sample(vacancies, 4)  # получили набор из случайных вакансий
        add_data_to_collection(vacancies, sample)  # попытались добавить набор в коллекцию
        pprint_cursor_object(salary_filter(vacancies, 500000))  # фильтруем и выводим вакансии


if __name__ == '__main__':
    main()
    
#************************************

# Mongo_queries.py

# Запросы к базе монго
#  суть зпроса ограничивается одной строчкой 
#  '$match':  # Фильтрация
#               {
#                   '$or': [{'salary_min_rub': {option: value}}, {'salary_max_rub': {option: value}}]
#
# Либо 'salary_min_rub' больше введёного значения 'option: value' 
# либо 'salary_max_rub' больше введёного значения 'option: value'
# с помощтю оператора '$or' это достигается. 

def salary_filter(collection, value, option='$gte'):
    """ Переводит валюту в рубли, создаёт соответствующие поля и фильтрует результат """

    return collection.aggregate([
        {
            '$fill':  # Заполнение None
                {
                    'output':
                        {
                            'salary_min': {'value': 0},
                            'salary_max': {'value': 0},
                        }
                }
        },
        {
            '$addFields':  # Конвертация валюты
                {
                    'salary_min_rub': {'$switch': {
                        'branches': [
                            {'case': {'$eq': ['$salary_currency', 'бел.руб.']}, 'then':
                                {'$multiply': ['$salary_min', 30]}},
                            {'case': {'$eq': ['$salary_currency', 'руб.']}, 'then':
                                {'$multiply': ['$salary_min', 1]}},
                            {'case': {'$eq': ['$salary_currency', 'USD']}, 'then':
                                {'$multiply': ['$salary_min', 65]}},
                            {'case': {'$eq': ['$salary_currency', 'EUR']}, 'then':
                                {'$multiply': ['$salary_min', 70]}},
                            {'case': {'$eq': ['$salary_currency', 'сум']}, 'then':
                                {'$multiply': ['$salary_min', 0.0053]}},
                            {'case': {'$eq': ['$salary_currency', 'KZT']}, 'then':
                                {'$multiply': ['$salary_min', 0.14]}}
                        ],
                        'default': None}},
                    'salary_max_rub': {'$switch': {
                        'branches': [
                            {'case': {'$eq': ['$salary_currency', 'бел.руб.']}, 'then':
                                {'$multiply': ['$salary_max', 30]}},
                            {'case': {'$eq': ['$salary_currency', 'руб.']}, 'then':
                                {'$multiply': ['$salary_max', 1]}},
                            {'case': {'$eq': ['$salary_currency', 'USD']}, 'then':
                                {'$multiply': ['$salary_max', 65]}},
                            {'case': {'$eq': ['$salary_currency', 'EUR']}, 'then':
                                {'$multiply': ['$salary_max', 70]}},
                            {'case': {'$eq': ['$salary_currency', 'сум']}, 'then':
                                {'$multiply': ['$salary_max', 0.0053]}},
                            {'case': {'$eq': ['$salary_currency', 'KZT']}, 'then':
                                {'$multiply': ['$salary_max', 0.14]}}
                        ],
                        'default': None}}}},
        {
            '$match':  # Фильтрация
                {
                    '$or': [{'salary_min_rub': {option: value}}, {'salary_max_rub': {option: value}}]
                }
        }])
#************************************

# Parcer_hh.py

""" Немного изменённый код из 2-ой домашней работы """

from bs4 import BeautifulSoup as bs
import requests


def salary_extraction(vacancy_salary):
    salary_dict = {'min': None, 'max': None, 'cur': None}

    if vacancy_salary:
        raw_salary = vacancy_salary.getText().replace(' – ', ' ').replace(' ', '').split()
        if raw_salary[0] == 'до':
            # до 380 000 руб.
            salary_dict['max'] = int(raw_salary[1])
        elif raw_salary[0] == 'от':
            # от 50 000 руб.
            salary_dict['min'] = int(raw_salary[1])
        else:
            # 50 000 – 100 000 руб.
            salary_dict['min'] = int(raw_salary[0])
            salary_dict['max'] = int(raw_salary[1])
        salary_dict['cur'] = ''.join(raw_salary[2:])  # решение проблемы бел. руб.

    return salary_dict


def get_vacancies(text):
    main_url = 'https://spb.hh.ru/'
    page_link = '/search/vacancy'  # ссылка на первую страницу поиска

    params = {'search_field': ['name', 'company_name', 'description'], 'items_on_page': 20,
              'text': text}
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                             'Chrome/101.0.4951.67 Safari/537.36'}

    vacancies = []
    while True:
        response = requests.get(main_url+page_link,
                                params=params,
                                headers=headers)
        html = response.text
        soup = bs(html, 'html.parser')
        vacancies_soup = soup.find_all('div', {'class': ['vacancy-serp-item-body__main-info']})

        for vacancy in vacancies_soup:

            vacancy_data = {'website': 'hh.ru'}

            vacancy_title = vacancy.find('a')
            vacancy_name = vacancy_title.getText()
            vacancy_link = vacancy_title['href'][: vacancy_title['href'].index('?')]
            vacancy_salary = salary_extraction(vacancy.find('span', {'class': ['bloko-header-section-3']}))
            try:
                vacancy_employer = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).getText().replace('\xa0', ' ')
                vacancy_address = vacancy.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).getText().replace('\xa0', ' ')
            except AttributeError:
                continue

            vacancy_data['name'] = vacancy_name
            vacancy_data['link'] = vacancy_link
            vacancy_data['salary_min'] = vacancy_salary['min']
            vacancy_data['salary_max'] = vacancy_salary['max']
            vacancy_data['salary_currency'] = vacancy_salary['cur']
            vacancy_data['employer'] = vacancy_employer
            vacancy_data['address'] = vacancy_address

            vacancies.append(vacancy_data)

        next_page = soup.find('a', {'data-qa': 'pager-next'})
        if not next_page:
            break

        page_link = next_page['href']
    return vacancies


if __name__ == '__main__':
    print(len(get_vacancies('data scientist')))

##### Комментарий от преподавателя 

In [ ]:
if collection.find_one('link', doc['link']):
    print('Документ существует в базе')

Для разбора другой работы. про используемую функцию `update_one({'ссылка': vac['ссылка']},{'$setOnInsert': {словарь того что вставляем}} , upsert=True)` 

если документ по критерию будет найден `({'ссылка': vac['ссылка']})`  Апдейт уан записывает на старые заниси новые. 

### https://github.com/LittleFox26

In [ ]:
from pymongo import MongoClient
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint

client = MongoClient('127.0.0.1', 27017)

db = client['vacDB']
# db.drop_collection(db.vacDB)
vacDB = db.vacDB

# https://hh.ru/search/vacancy?text=Data+scientist&area=1&salary=&currency_code=RUR&experience=doesNotMatter&order_by=relevance&search_period=0&items_on_page=50&no_magic=true&L_save_area=true&from=suggest_post
main_url = 'https://hh.ru'
vacancy = 'Data Scientist'
page = 0
all_vacancies = []
params = {'text': vacancy,
          'area': 2,
          'experience': 'doesNotMatter',
          'order_by': 'relevance',
          'search_period': 0,
          'items_on_page': 19,
          'page': page}
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
                         'AppleWebKit/537.36 (KHTML, like Gecko)'
                         'Chrome/98.0.4758.141 YaBrowser/22.3.4.731 Yowser/2.5 Safari/537.36'}
response = requests.get(main_url + '/search/vacancy', params=params, headers=headers)

while True:

    with open('page.html', 'w', encoding='utf-8') as f:
        f.write(response.text)

    with open('page.html', 'r', encoding='utf-8') as f:
        html = f.read()

    soup = bs(html, 'html.parser')

    vacancies = soup.find_all('div', {'class': 'vacancy-serp-item'})

    for vacancy in vacancies:

        vacancy_info = {}
        vacancy_anchor = vacancy.find('a', {'data-qa': "vacancy-serp__vacancy-title"})
        vacancy_name = vacancy_anchor.getText()
        vacancy_info['name'] = vacancy_name

        vacancy_link = vacancy_anchor['href']
        vacancy_info['link'] = vacancy_link

        vacancy_info['site'] = main_url + '/'

        vacancy_salary = vacancy.find('span', {'data-qa': "vacancy-serp__vacancy-compensation"})
        if vacancy_salary is None:
            min_salary = None
            max_salary = None
            currency = None
        else:
            vacancy_salary = vacancy_salary.getText()
            if vacancy_salary.startswith('до'):
                max_salary = int("".join([s for s in vacancy_salary.split() if s.isdigit()]))
                min_salary = None
                currency = vacancy_salary.split()[-1]

            elif vacancy_salary.startswith('от'):
                max_salary = None
                min_salary = int("".join([s for s in vacancy_salary.split() if s.isdigit()]))
                currency = vacancy_salary.split()[-1]

            else:
                max_salary = int("".join([s for s in vacancy_salary.split('–')[1] if s.isdigit()]))
                min_salary = int("".join([s for s in vacancy_salary.split('–')[0] if s.isdigit()]))
                currency = vacancy_salary.split()[-1]

        vacancy_info['max_salary'] = max_salary
        vacancy_info['min_salary'] = min_salary
        vacancy_info['currency'] = currency

        all_vacancies.append(vacancy_info)

    next_button = soup.find('a', {'data-qa': "pager-next"})
    if next_button is None:
        break
    else:
        response = requests.get(main_url + next_button['href'], headers=headers)

print(len(all_vacancies))
# pprint(all_vacancies)

# ДОБАВЛЕНИЕ ТОЛЬКО НОВЫХ ВАКАНСИЙ В ДБ:


def db_update(database, vac_list):

# ***************************************************************************
# Комментарий преподавателя по функции ниже:

# Запрос к базе данных делается безусловный это ПЛОХО 
# database.find({}) - мы всю массу которая у нас лежит мы поднимаем наверх
# в медленные тормознутые пайтоновские списки. 
# И эти списки ещё начинаем обрабатывать по нескольким значениям
# el['name'], el['max_salary'] и т.д.
# Нам наружу поднимать не надо всё, а лишь поднять нужный результат. 


    x = True
    for vac in vac_list:
        for el in database.find({}):
            if el['name'] == vac['name'] and \
               el['max_salary'] == vac['max_salary'] and \
               el['min_salary'] == vac['min_salary'] and \
               el['currency'] == vac['currency']:
                x = False
                break
            else:
                x = True
        if x:
            database.insert_one(vac)
    return database


db_update(vacDB, all_vacancies)
# for i in vacDB.find({}):
#     print(i)

# Подбор вакансий из ДБ по ЗП:


def vacancy_by_salary(database):

    try:
        salary = int(input('Insert desired salary: '))
        s = []
        for el in database.find({'$and': [
            {'$or': [{'min_salary': {'$type': 'number'}}, {'max_salary': {'$type': 'number'}}]},
            {'$or': [{'min_salary': {'$gt': salary}}, {'max_salary': {'$gt': salary}}]}
                                         ]}):
            s.append(el)
        return pprint(s)
    except ValueError:
        print('Salary should be an integer number.')


vacancy_by_salary(vacDB)

### https://github.com/Templl

In [ ]:
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke

from pprint import pprint
import json
import re

#загружаем вакансии
with open('data.txt') as json_file:
    data = json.load(json_file)

#добавляем индетификатор вакансии
for vacancy in data:
    link = vacancy['link']
    id = re.split('/', link)[4]
    vacancy['_id'] = id[:8]


###
#подключаемся к базе монго
client = MongoClient('127.0.0.1', 27017)
db = client['vacncy2005']  # база данных
vacancy = db.vacancy  # название коллекции

#загружаем в базу вакансии
for i in range(len(data)):
    try:
        vacancy.insert_one(data[i])
    except dke:
        #print(f'Вакансия "{data[i]["name"]}" уже есть в базе')
        

        
        
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke

from pprint import pprint

#current_course =

#подключаемся к базе монго
client = MongoClient('127.0.0.1', 27017)
db = client['vacncy2005']  # база данных
vacancy = db.vacancy  # название коллекции

# вводе переменных для поиска
salary = input('Желаемая зарплата:')
currency = input('В какой валюте зарплата:')

#salary = 220000
#currency = 'руб.'

#result = list(vacancy.find({}))
#pprint(result)

for doc in vacancy.find(
        {'мин': {'$gte': salary},
         'макс': {'$gte': salary},
         'валюта': {'$eq': currency}}
):
    if doc == None:
        print('вакансии с такими параметрами не найдены')
    else:
        pprint(doc)

result = list(vacancy.find(
                            {'мин': {'$gte': salary},
                             'макс': {'$gte': salary},
                             'валюта': {'$eq': currency}}
))

if len(result):
    pprint(result)
else:
    print('Подходящих вакансий не найдено')

Есть метод подсчитывающий найденные документы (количество найденных документов): persons.count_documrnts({'age': {'$gte': 30}})

### собрать 5 новостей с новостного ресурса например https://ria.ru

Ищу контейнер со всеми пятью новостями через инструмент разработчика:
`<div class="cell-list__list">` и через селектор  XPATH:
`//div[@class="cell-list__list"]`

In [43]:
from lxml import html
import requests
from pprint import pprint

url_ria = 'https://ria.ru/'
headers = {'User Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}

response = requests.get(url_ria, headers=headers)

dom_ria = html.fromstring(response.text)

news = dom_ria.xpath("//span[@class='cell-list__item-title']/text()")

In [44]:
response

<Response [200]>

In [ ]:
dom_ria = html.fromstring(response.text)

Посмотрел через инструмент разработчика интересующий элемент и составил икс пас запрос

In [ ]:
news = dom_ria.xpath("//span[@class='cell-list__item-title']/text()")

Вывожу первые 5 новостей

In [56]:
news[:5] 

['Число жертв землетрясений в Турции превысило 16,5 тысячи ',
 'Офис президента Украины отредактировал пост о скорых поставках истребителей ',
 'МИД назвал реакцию Запада на очередной расстрел российских пленных постыдной ',
 'На Купянском и Краснолиманском направлениях уничтожили более 140 украинских солдат ',
 'В Новосибирске задержали сотрудников газовой компании, осматривавших дом перед взрывом ']

In [57]:
len(news)

29

In [19]:
from lxml import html
import requests
from pprint import pprint

url = 'https://www.thesun.co.uk/'
headers = {'User Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'}

response = requests.get(url, headers=headers)

dom = html.fromstring(response.text)

news = dom.xpath("//a[@class='teaser-anchor']/data-headline")



In [20]:
response

<Response [200]>

In [21]:
news

[]